Rosita Raišuotytė vgg16 ["Bee", "Castle", "Banana"]

# Nuotraukų įsikėlimas


In [ ]:
pip install openimages

In [ ]:
import os
from openimages.download import download_dataset

In [ ]:
data_dir = "OpenImages"
number_for_samples = 372
classes = ["Bee", "Castle", "Banana"]

In [ ]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
print("Downloading is starting...")
download_dataset(data_dir, classes, limit=number_for_samples)

# Gpu

In [ ]:
import torch

In [ ]:
print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Modelio sukurimas

In [ ]:
from torchvision import models

In [ ]:
model = models.vgg16(pretrained = True).to(device)
model.eval()

# Transformacijos

In [ ]:
from torchvision import transforms

In [ ]:
all_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Dataset

In [ ]:
from torch.utils.data import Dataset

In [ ]:
import torch
from pandas.core.common import flatten
import numpy as np

from PIL import Image
from typing import Tuple, Dict, List

import glob

In [ ]:
data_path = 'OpenImages'

image_paths = []      #to store image paths in list
image_classes = []    #to store class values

for data_path in glob.glob(data_path + '/*'):
    image_classes.append(data_path.split('/')[-1])
    image_paths.append(glob.glob(data_path + "/images" +  '/*'))
    
image_paths = list(flatten(image_paths))

In [ ]:
# Pagalbinės funkcijos verčiant pavadinimą (label) į indeksą
idx_to_class = {i:j for i, j in enumerate(image_classes)}
class_to_idx = {value:key for key, value in idx_to_class.items()}
print(idx_to_class)
print(class_to_idx)

In [ ]:
class MyDataset(Dataset):

   def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
   def __len__(self):
        return len(self.image_paths)

   def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = Image.open(image_filepath)
        if image.mode is 'L':
            image = image.convert('RGB')
        image = self.transform(image)
        
        label = image_filepath.split('/')[1]
        label = class_to_idx[label]
        
        return image, label

In [ ]:
dataset = MyDataset(image_paths, all_transforms)

#  DataLoader

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
loader = DataLoader(
    dataset, batch_size=32, shuffle=True, num_workers=2
)

iterator = iter(loader)

# While ciklas kol pereisim visas nuotraukas

In [ ]:
ground_truth = []
predictions_bee = []
predictions_banana = []
predictions_castle = []

while True:
        try:
            features, labels = next(iterator)
            output = model(features.to(device))

            for i in range(output.shape[0]):
              predictions = torch.softmax(output[i], 0)
              predictions_castle = np.append(predictions_castle, predictions[483].cpu().detach())
              predictions_bee = np.append(predictions_bee, predictions[309].cpu().detach())
              predictions_banana = np.append(predictions_banana, predictions[954].cpu().detach())

            ground_truth = np.append(ground_truth, labels)
        except StopIteration:
            break

# Skaičiuojame lenteles funkcija

In [ ]:
def calculate_confusion_matrix(ground_truth, predictions, class_idx, threshold = 0.5):
  predictions_thresholded = (predictions >= threshold).astype(np.float64)
  matrix = {}
  matrix['TP'] = np.sum(np.bitwise_and(ground_truth == class_idx, predictions_thresholded == 1))
  matrix['TN'] = np.sum(np.bitwise_and(ground_truth != class_idx, predictions_thresholded == 0))
  matrix['FP'] = np.sum(np.bitwise_and(ground_truth != class_idx, predictions_thresholded == 1))
  matrix['FN'] = np.sum(np.bitwise_and(ground_truth == class_idx, predictions_thresholded == 0))
  # print(matrix['TN'], matrix['FP'], matrix['FN'], matrix['TP'])

  return matrix

  # Confusion matrix
  #   0  1 predicted 
  # 0 TN FP
  # 1 FN TP

# Skaičiuojame metrikas funkcija

In [ ]:
def calculate_metrics(TP, TN, FP, FN):
  metrics = {}
  metrics['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
  metrics['recall'] = TP / (TP + FN)
  metrics['precision'] = TP / (TP + FP)
  metrics['f1'] = 2 * (metrics['recall'] * metrics['precision']) / (metrics['recall'] + metrics['precision'])

  return metrics

# Atliekame skaičiavimus

In [ ]:
matrix_banana = calculate_confusion_matrix(ground_truth, predictions_banana, 0, 0.05)
matrix_bee = calculate_confusion_matrix(ground_truth, predictions_bee, 1, 0.05)
matrix_castle = calculate_confusion_matrix(ground_truth, predictions_castle, 2, 0.05)

matrix_all = {}
matrix_all['TP'] = matrix_banana['TP'] + matrix_bee['TP'] + matrix_castle['TP']
matrix_all['TN'] = matrix_banana['TN'] + matrix_bee['TN'] + matrix_castle['TN']
matrix_all['FP'] = matrix_banana['FP'] + matrix_bee['FP'] + matrix_castle['FP']
matrix_all['FN'] = matrix_banana['FN'] + matrix_bee['FN'] + matrix_castle['FN']

metrics_banana = calculate_metrics(matrix_banana['TP'], matrix_banana['TN'], matrix_banana['FP'], matrix_banana['FN'])
metrics_bee = calculate_metrics(matrix_bee['TP'], matrix_bee['TN'], matrix_bee['FP'], matrix_bee['FN'])
metrics_castle = calculate_metrics(matrix_castle['TP'], matrix_castle['TN'], matrix_castle['FP'], matrix_castle['FN'])
metrics_all = calculate_metrics(matrix_all['TP'], matrix_all['TN'], matrix_all['FP'], matrix_all['FN'])

# Spausdinimo fukcija ir spausdinimas

In [ ]:
def print_metrics(metrics, class_idx):
  print('Class ', idx_to_class[class_idx], ' metrics:')
  print('  accuracy : ', metrics['accuracy'])
  print('  recall : ', metrics['recall'])
  print('  precision : ', metrics['precision'])
  print('  f1 : ', metrics['f1'])
  print()

In [ ]:
def print_all_metrics(metrics):
  print('All ', ' metrics:')
  print('  accuracy : ', metrics['accuracy'])
  print('  recall : ', metrics['recall'])
  print('  precision : ', metrics['precision'])
  print('  f1 : ', metrics['f1'])

In [42]:
print_metrics(metrics_banana, 0)
print_metrics(metrics_bee, 1)
print_metrics(metrics_castle, 2)
print_all_metrics(metrics_all)

Class  banana  metrics:
  accuracy :  0.9080919080919081
  recall :  0.7526881720430108
  precision :  1.0
  f1 :  0.8588957055214724

Class  bee  metrics:
  accuracy :  0.8901098901098901
  recall :  0.7043010752688172
  precision :  1.0
  f1 :  0.8264984227129338

Class  castle  metrics:
  accuracy :  0.972027972027972
  recall :  0.8910505836575876
  precision :  1.0
  f1 :  0.9423868312757202

All   metrics:
  accuracy :  0.9234099234099234
  recall :  0.7702297702297702
  precision :  1.0
  f1 :  0.8702031602708804


# Prireikė dirbant...

In [ ]:
# Labels indexai modelio output'e
with open('imagenet_classes.txt', 'r') as fid:
  imagenet_labels = [ln.strip() for ln in fid]
print(imagenet_labels)
print(imagenet_labels.index("bee"))
print(imagenet_labels.index("banana"))
print(imagenet_labels.index("castle"))

In [ ]:
torch.cuda.empty_cache()